**Importação das bibliotecas**

In [1]:
import numpy as np
from scipy.linalg import lu

**Funções Auxiliares**

In [4]:
def substituicao_direta(L, b):
  """
  Resolve o sistema triangular inferior Ly = b para y (Forward Substitution).
  L tem 1s na diagonal - Método Doolittle.
  """
  n = L.shape[0]
  y = np.zeros(n, dtype=float)

  for i in range(n):
    sum_j = 0
    for j in range(i):
      sum_j += L[i, j] * y[j]
      y[i] = b[i] - sum_j

    return y

def substituicao_reversa(U, y):
  """
  Resolve o sistema triangular superior Ux = y para x (Backward substitution).
  """
  n = U.shape[0]
  x = np.zeros(n, dtype=float)

  for i in range(n - 1, -1, -1):
    sum_j = 0
    for j in range(i + 1, n):
      sum_j += U[i, j] * x[j]

    if np.isclose(U[i, i], 0):
      raise ValueError("Matriz U é singular, o sistema pode não ter solução única.")

    x[i] = (y[i] - sum_j) / U[i, i]

  return x

**Funções Principais**

In [5]:
def resolve_lu(A_in, b_in):
  """
  Implementa as três etapas do processo de decomposição LU
  """
  A = np.array(A_in, dtype=float)
  b = np.array(b_in, dtype=float)

  P, L, U = lu(A)

  Pb = P @ b
  y = substituicao_direta(L, Pb)
  x = substituicao_reversa(U, y)

  return P, L, U, y, x